# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [1]:
from selenium import webdriver
#importampos libreria para cargar el driver automaticamente
from webdriver_manager.firefox import GeckoDriverManager
import time

url='https://www.latamairlines.com/py/es/ofertas-vuelos?origin=ASU&outbound=2023-08-01T12%3A00%3A00.000Z&destination=BCN&inbound=null&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

In [2]:
def obtener_precios(vuelo):
    '''
    Función que retorna una lista de diccionarios con las distintas tarifas
    '''
    tarifas= vuelo.find_elements('xpath','.//ol[@class="sc-buGlAa jhwXGF"]/li[@class="sc-kecUPG dPNrrD"]')
    precios=[]
    for tarifa in tarifas:
        #buscamos en cada pocision de la tarifa los siquientes elementos
        nombre = tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[1]/span[@class="sc-fhiYOA iwcbaW"]').text
        moneda= tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[contains(@class,"currency")]').text
        valor= tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[@class="sc-ckYZGd grNCid"]').text
        
        #guardo los valores que obtengo en un diccionario
        dict_tarifa={nombre:{'moneda':moneda,'valor':valor}}
        
        #guardamos nuestro diccionario con los datos de tarifa a nuestra lista de precio
        precios.append(dict_tarifa)
    return precios

In [3]:
def obtener_datos_escalas(vuelo):
    '''
    Función que retorna una lista de diccionarios con la información de 
    las escalas de cada vuelo
    '''
    # seleccionamos los elementos pertencientes a los vuelos antes de las escalas
    segmentos= vuelo.find_elements('xpath','//section[@class="sc-fGSyRc fCuylQ"]')
    datos_escalas=[]
    duracion_escalas_dic={}    
    for segmento in segmentos:
        #origen
        salida=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[1]').text
        #hora de salida
        hora_salida=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[2]').text
        #destino
        destino_segmento=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]//div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[1]').text
        #Hora de llegada
        hora_llegada=segmento.find_element('xpath','.//div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[2]').text
        #Duracion del vuelo
        duracion_segmento=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]//div[@class="sc-ewMkZo hQNSAX"]/span[2]').text
        #Numero de vuelo
        numero_vuelo_segmento=segmento.find_element('xpath','.//div[@class="sc-dzQEYZ dslPlz airline-wrapper"]').text
        #Modelo de avion
        modelo_avion_segmento=segmento.find_element('xpath','.//div[@class="sc-sVRsr eXYUTi"]//span[@class="airplane-code"]').text
        
        # Armo un diccionario para almacenar los datos
        datos_escalas_dict={
            'salida':salida,
            'hora_salida':hora_salida,
            'destino':destino_segmento,
            'hora_llega':hora_llegada,
            'duracion':duracion_segmento,
            #'duracion_escala':duracion_escala_vuelo,
            'numero_vuelo':numero_vuelo_segmento,
            'modelo_avion':modelo_avion_segmento
        }
        datos_escalas.append(datos_escalas_dict)
    
    # seleccionamos los segmentos pertencientes a las escalas
    escalas_vuelo=vuelo.find_elements('xpath','//section[@class="sc-kiXyGy sc-eZXMBi dKgCnQ connectionInfo"]')
    for num_escala, escala in enumerate(escalas_vuelo):
        escala=escala
        #conexion de la escala
        escala_vuelo=escala.find_element('xpath','.//div[@class="sc-ekQYnd cByWfv"]//span[@class="connection-text"]').text
        #duracion de la escala
        duracion_escala_vuelo=escala.find_element('xpath','.//div[@class="sc-ekQYnd cByWfv"]//span[@class="time"]').text
        # guardamos en un diccionario la escala actual
        duracion_escalas={f'Escala {num_escala+1}':escala_vuelo,f'Duracion Escala {num_escala+1}':duracion_escala_vuelo}
        # actualizamos nuestro dicionario de escalas totales
        duracion_escalas_dic.update(duracion_escalas)

    # agregamos nuestras escalas totales en nuestra lista de datos    
    datos_escalas.append({'duracion_escalas':duracion_escalas_dic})

    return datos_escalas

In [4]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada
    vuelo, incluyendo la duración. 
    Nota: la duración del vuelo no es la hora de llegada - la hora de salida porque
    puede haber diferencia de horarios entre el origen y el destino.
    '''
    #hora de salida
    hora_salida=vuelo.find_element('xpath','.//div[@class="sc-klSiHT hjzFuR flight-information"]/span[1]').text
    #hora de llegada
    hora_llegada=vuelo.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
    # Duracion del vuelo
    duracion_vuelo= vuelo.find_element('xpath','.//div[2]/span[2]').text

    tiempos_vuelo_dict={
        'hora_salida':hora_salida,
        'hora_llegada':hora_llegada,
        'duracion_vuelo':duracion_vuelo
    }
    return tiempos_vuelo_dict

In [7]:
def obtener_info(driver):
    #Usaremos el Xpath para obtener la lista de vuelos
    vuelos = driver.find_elements('xpath','//ol/li[@class="sc-bvTASY cfqKKq"]')
    print(f'Se encontaron {len(vuelos)} vuelos.')
    print(f'Iniciando Scraping...')
    info=[]
    contador=0
    for vuelo in vuelos:
        contador+=1
        print(f'Analizando {contador} de {len(vuelos)} vuelos')
        #obtener los tiempos generales de cada vuelo
        tiempos=obtener_tiempos(vuelo)
        #clickeamos sobe el link escalas
        vuelo.find_element('xpath','.//div[@class="sc-iKiVwC fbWfQZ"]//a').click()
        #esperamos que cargue el contenido
        time.sleep(10)
        escalas=obtener_datos_escalas(vuelo)
        #cerramos el modal
        driver.find_element('xpath','//div[@class="MuiDialog-container MuiDialog-scrollPaper"]').click()
        time.sleep(10)
        #clickeamos el vuelo para ver los precios
        vuelo.find_element('xpath','.//div[@class="sc-cIbcTr iuAYhf"]').click()
        # vuelo.click()
        #esperamos que cargue el contenidon de los precios
        time.sleep(10)
        precios=obtener_precios(vuelo)
        #click nuevamente para cerrar los precios que se desplegaron
        vuelo.find_element('xpath','.//div[@id="undefined--button-wrapper"]//button').click()

        info.append({'precios':precios,'tiempos':tiempos,'escalas':escalas})

    return info

In [8]:
options = webdriver.FirefoxOptions()
# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('-private')
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)
driver.get(url)
#si aparece un mensaje popups apenas inicie la web
def close_dialog():
    '''
    Función que cierra los dialog o popups de la página
    '''
    dialogs = driver.find_elements('xpath', '//div[contains(@class,"MuiDialog-container")]')
    if len(dialogs):
        dialogs[0].find_element('xpath', './/button[contains(@class,"Dialog__CloseButton")]').click()

close_dialog()
time.sleep(10)
obtener_info(driver)

[WDM] - Downloading: 19.0kB [00:00, 6.52MB/s]                   
C:\Users\mario\AppData\Local\Temp\ipykernel_23504\821188174.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


Se encontaron 27 vuelos.
Iniciando Scraping...
Analizando 1 de 27 vuelos
Analizando 2 de 27 vuelos
Analizando 3 de 27 vuelos
Analizando 4 de 27 vuelos
Analizando 5 de 27 vuelos
Analizando 6 de 27 vuelos
Analizando 7 de 27 vuelos
Analizando 8 de 27 vuelos
Analizando 9 de 27 vuelos
Analizando 10 de 27 vuelos
Analizando 11 de 27 vuelos
Analizando 12 de 27 vuelos
Analizando 13 de 27 vuelos
Analizando 14 de 27 vuelos
Analizando 15 de 27 vuelos
Analizando 16 de 27 vuelos
Analizando 17 de 27 vuelos
Analizando 18 de 27 vuelos
Analizando 19 de 27 vuelos
Analizando 20 de 27 vuelos
Analizando 21 de 27 vuelos
Analizando 22 de 27 vuelos
Analizando 23 de 27 vuelos
Analizando 24 de 27 vuelos
Analizando 25 de 27 vuelos
Analizando 26 de 27 vuelos
Analizando 27 de 27 vuelos


[{'precios': [{'plus': {'moneda': 'USD', 'valor': '1,341.30'}},
   {'top': {'moneda': 'USD', 'valor': '3,153.30'}}],
  'tiempos': {'hora_salida': '6:55 a. m.',
   'hora_llegada': '5:20 p. m.',
   'duracion_vuelo': '28 h 25 min'},
  'escalas': [{'salida': 'ASU',
    'hora_salida': '6:55 a. m.',
    'destino': 'LIM',
    'hora_llega': '10:00 a. m.',
    'duracion': '4 h 5 min',
    'numero_vuelo': 'LA1320',
    'modelo_avion': 'Airbus A320'},
   {'salida': 'LIM',
    'hora_salida': '8:20 p. m.',
    'destino': 'MAD',
    'hora_llega': '2:35 p. m.',
    'duracion': '11 h 15 min',
    'numero_vuelo': 'LA5391',
    'modelo_avion': 'Airbus A350-900'},
   {'salida': 'MAD',
    'hora_salida': '4:00 p. m.',
    'destino': 'BCN',
    'hora_llega': '5:20 p. m.',
    'duracion': '1 h 20 min',
    'numero_vuelo': 'LA1523',
    'modelo_avion': 'Airbus A319'},
   {'duracion_escalas': {'Escala 1': 'Conexión Lima',
     'Duracion Escala 1': '10 h 20 min',
     'Escala 2': 'Conexión Madrid',
     'Durac

Paso 4: cerrar el navegador

In [9]:
driver.close()